In [ ]:
import requests
import json
import pandas as pd
import xmltodict
from datetime import date,timedelta

In [ ]:
def load_asos(fname):
    """
    Load the asos codes.
        Args:
            fname (str) : ASOS code file name(csv)
        Returns:
            codes (list) : ASOS code list including the dict
    """
    asos_code = pd.read_csv(fname,encoding='utf-8')
    codes = [{c:n} for c,n in zip(asos_code['Code'], asos_code['name'])]
    return codes

In [ ]:
def call_asosRN(codes, params, api_url, mode='0'):
    """
    Call the API
        Args:
            codes (list) : ASOS code list including the dict
            params (dict) : The parameters for the API requests
            api_url (str) : API url
            mode (str) : The mode 0 is for the day unit API and the mode 1 is for the hour unit API
        Returns:
            asos (dict) : The dictionary including the API responses
    """
    tmp = []
    for c in codes:
        stnIds = list(c.keys())[0]
        params['stnIds'] = c

        # API request
        response = requests.get(day_api_url, params=params)
        
        # XML to DICT
        xmlToDict = xmltodict.parse(response.text)
        res = json.loads(json.dumps(xmlToDict))

        # Normal service
        if 'response' in res.keys():
            if res['response']['header']['resultCode']=='00':
                # Hours
                if mode=='1':
                    for item in res['response']['body']['items']['item']:
                        tmp.append({'STN_ID' : item['stnId'],'STN_NAME': item['stnNm'],'YMDH':item['tm'],'RN_HR' : item['rn']})
                # Days
                elif mode=='0':
                    item = res['response']['body']['items']['item']
                    tmp.append({'STN_ID' : item['stnId'],'STN_NAME': item['stnNm'],'YMD':item['tm'],'RN_DAY' : item['sumRn']})
                        
            else:
                pass

        # API service errors
        elif 'OpenAPI_ServiceResponse' in res.keys():
            break
    asos = {'results' : ''}
    if len(tmp)!=0:
        asos['results'] = tmp 
        
    return asos


In [ ]:
if __name__ == "__main__":
    codes = load_asos('./ASOS_code.csv')
    dates = str(date.today() - timedelta(1)).replace('-','')

    apiKey = 'apiKey'
    hr_api_url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
    day_api_url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

    pageNo = '1'
    numOfRows = '24'
    dataType = 'XML'
    # Data code
    dataCd = 'ASOS'
    # Start Date
    startDt = dates
    # End Date
    endDt = dates
    # Start hour
    startHh = '00'
    # End horu
    endHh = '23'
    
    hr_params = {'ServiceKey' : apiKey,
                  'pageNo' : pageNo,
                  'numOfRows' : numOfRows,
                  'dataType' : dataType,
                  'dataCd' : dataCd,
                  'dateCd' : 'HR',
                  'startDt' : dates,
                  'endDt' : dates,
                  'startHh' : startHh,
                  'endHh' : endHh}

    day_params = {'ServiceKey' : apiKey,
              'pageNo' : pageNo,
              'numOfRows' : numOfRows,
              'dataType' : dataType,
              'dataCd' : dataCd,
              'dateCd' : 'DAY',
              'startDt' : dates,
              'endDt' : dates}
    


    hr_df = call_asosRN(codes, hr_params, hr_api_url, mode='1')
    day_df = call_asosRN(codes, day_params, day_api_url, mode='0')
    


In [ ]:
day_df
